In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ['HF_HOME'] = '/shared/data3/pk36/.cache'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
!export HF_HOME=/shared/data3/pk36/.cache
os.environ['CUDA_VISIBLE_DEVICES'] = "6,7"

In [3]:
from debate import DebateNode
from paper_details import Paper
from persona import PaperAuthor
from moderator import Moderator
import argparse
from typing import List
from vllm import LLM
import os
import json
from data_pairer import parse_papers
import numpy as np

In [4]:
class Args():
    def __init__(self):
        self.log_dir = "logs"
        self.focus_paper = "https://arxiv.org/pdf/1706.03762"
        self.cited_paper = "https://arxiv.org/pdf/1810.04805"
        self.topic = "language model architectures"

args = Args()

In [5]:
if not os.path.exists(args.log_dir):
    os.makedirs(args.log_dir)

# if not os.path.exists("data.json"):
parse_papers(args.focus_paper, args.cited_paper)
with open('data.json', 'r') as file:
    data = json.load(file)

In [6]:
model_server = LLM(model="meta-llama/Meta-Llama-3.1-8B-Instruct",tensor_parallel_size=2, gpu_memory_utilization=0.5, max_num_seqs=100) #,enable_prefix_caching=True)

INFO 11-23 07:02:12 config.py:905] Defaulting to use mp for distributed inference
WARNING 11-23 07:02:12 arg_utils.py:957] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 11-23 07:02:12 config.py:1021] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 11-23 07:02:12 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


(VllmWorkerProcess pid=132748) INFO 11-23 07:02:18 model_runner.py:1067] Loading model weights took 7.5122 GB
INFO 11-23 07:02:18 model_runner.py:1067] Loading model weights took 7.5122 GB
INFO 11-23 07:02:19 distributed_gpu_executor.py:57] # GPU blocks: 15590, # CPU blocks: 4096
INFO 11-23 07:02:19 distributed_gpu_executor.py:61] Maximum concurrency for 131072 tokens per request: 1.90x
INFO 11-23 07:02:21 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=132748) INFO 11-23 07:02:21 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-23 07:02:21 model_runner.py:1395] Capturing the mode

In [7]:
item = data[0]
f_pap = item['focus']
c_pap = item['cited']

focus_paper = PaperAuthor(
    model = model_server,
    paper = Paper(f_pap),
    focus=True,
    id=0
)

cited_paper = PaperAuthor(
    model = model_server,
    paper = Paper(c_pap),
    focus=False,
    id=1
)

100%|██████████| 2/2 [00:01<00:00,  1.62it/s]


In [9]:
focus_paper.paper.retrieve_top_k("language model architectures", k=5)

100%|██████████| 1/1 [00:00<00:00, 26.40it/s]


[('We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train',
  0.73787886),
 ('Introduction Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks in particular, have been firmly established as state of the art approaches in sequence modeling and transduction problems such as language modeling and machine translation [35, 2, 5]. Numerous efforts have since continued to push the boundaries of recurrent language models and encoder-decoder architectures [38, 24, 15]',
  0.73269325),
 ('We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data. *Equal contribution',
  0.7009047),
 ('Th

In [11]:
focus_paper.paper.retrieve_top_k("multi headed attention", k=5)

100%|██████████| 1/1 [00:00<00:00,  3.51it/s]


[('With a single attention head, averaging inhibits this. MultiHead(Q, K, V ) = Concat(head1, ..., headh)W O where headi = Attention(QW Q i , KW Ki , V W Vi ) Where the projections are parameter matrices W Q and W O Rhdvxdmodel',
  0.8372327),
 ('We inspect attention distributions from our models and present and discuss examples in the appendix. Not only do individual attention heads clearly learn to perform different tasks, many appear to exhibit behavior related to the syntactic and semantic structure of the sentences',
  0.7794119),
 ('These are concatenated and once again projected, resulting in the final values, as depicted in Figure 2. Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions',
  0.77801174),
 ('The first is a multi-head self-attention mechanism, and the second is a simple, positionwise fully connected feed-forward network. We employ a residual connection [11] around each of the two sub-l

In [29]:
moderator = Moderator(model_server)

paper_authors = [focus_paper, cited_paper]
leaf_node_label = args.topic

if args.log_dir != "":
    with open(os.path.join(args.log_dir, 'self_deliberation.txt'), 'w') as f:
        f.write(f'Topic: {args.topic}\n\n')

# each node has a topic
root_node = DebateNode(leaf_node_label)
subtrees = root_node.conduct_self_deliberation(leaf_node_label, paper_authors, log=args.log_dir) # k new, finer topics to discuss

100%|██████████| 1/1 [00:00<00:00, 41.32it/s]


In [30]:
root_node.self_delib

{0: [{'title': 'New Architecture Significantly Outperforms Traditional RNN Architectures',
   'description': 'Our Transformer architecture is more effective than traditional RNN models in two key translation tasks, demonstrating superiority in quality. We based our conclusion on experiments with two machine translation tasks showing that models are superior in quality while being more parallelizable and requiring significantly less time to train.'},
  {'title': 'The Transformer Helps Solve Major Challenges in Sequence Modelling Tasks',
   'description': 'The proposed Transformer architecture has generalizability skills helping the model complete tasks such as English constituency parsing even with limited data successfuly,We outshine with state-of-the-art results in such hard data regimes that other RNN sequence-to-sequence models are unable to reach.'}],
 1: [{'title': 'Advancements in Bidirectionality for Pre-training Language Models',
   'description': 'Our work offers a novel bidir

In [31]:
root_node.preemption

{0: {'Does my paper also address the claim, "advancements in bidirectionality for pre-training language models"?': ['We used a beam size of 21 and a = 0.3 for both WSJ only and the semi-supervised setting. Our results in Table 4 show that despite the lack of task-specific tuning our model performs surprisingly well, yielding better results than all previously reported models with the exception of the Recurrent Neural Network Grammar [8]',
   'In row (E) we replace our sinusoidal positional encoding with learned positional embeddings [9], and observe nearly identical results to the base model. 6.3 English Constituency Parsing To evaluate if the Transformer can generalize to other tasks we performed experiments on English constituency parsing'],
  'Does my paper also address the claim, "enriched pre-training objectives promote deeper representation learning"?': ['We also experimented with using learned positional embeddings [9] instead, and found that the two versions produced nearly ide

In [41]:
out = focus_paper.present_argument(root_node.children[0], root_node)

You are an author of a paper that is debating another author about the following topic:
	- Topic: New Architecture Significantly Outperforms Traditional RNN Architectures
	- Topic Description: Our Transformer architecture is more effective than traditional RNN models in two key translation tasks, demonstrating superiority in quality. We based our conclusion on experiments with two machine translation tasks showing that models are superior in quality while being more parallelizable and requiring significantly less time to train.

Your debate claim is that your paper's contributions towards the topic are all novel relative to the other paper. Refer to their arguments and presented evidence, as well as your own paper's segments as evidence when refining your arguments.

You used the following evidence to support your arguments:
	- Evidence #1. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions en

In [47]:
cited_out = cited_paper.present_argument(root_node.children[0], root_node)

You are an author of a paper that is debating another author about their claimed novelty:
	- Novelty Claim: New Architecture Significantly Outperforms Traditional RNN Architectures
	- Novelty Claim Description: Our Transformer architecture is more effective than traditional RNN models in two key translation tasks, demonstrating superiority in quality. We based our conclusion on experiments with two machine translation tasks showing that models are superior in quality while being more parallelizable and requiring significantly less time to train.

Your debate claim is that the other paper's contributions towards the topic are not novel relative to your own paper. Refer to their arguments and presented evidence, as well as your own paper's segments as evidence when refining your arguments.

You initially argued that your own paper has the following novelties:
	- Argument #1. Advancements in Bidirectionality for Pre-training Language Models: Our work offers a novel bidirectional pre-train

In [48]:
out

{'title': 'Consistent Superiority in Sequence Modeling Tasks for the Proposed Transformer Architecture.',
 'description': "Our paper's contributions are novel and significant in sequence modeling and transduction, consistently surpassing traditional RNN architectures in accuracy and efficiency. Through a rigorous critique of the opposing author's points of novelty and the evidence presented in our own paper, we demonstrate the superiority of the Transformer architecture over traditional RNN models in multiple machine translation tasks."}

In [ ]:
conversation_history = []

queue_of_rounds: List[DebateNode] = []
queue_of_rounds.extend(subtrees)


while len(queue_of_rounds) > 0:
    round = queue_of_rounds.pop(0)
    conversation, new_focus_arg, new_cited_arg = round.conduct_debate(focus_paper, cited_paper)
    conversation_history.extend(conversation)
    if moderator.is_expand(round.self_delib, [new_focus_arg, new_cited_arg]):
        new_subtrees = round.conduct_self_deliberation(round.round_topic, paper_authors)
        queue_of_rounds.extend(new_subtrees)

with open('conversation_history.txt', 'w+') as f:
    f.write('\n'.join(conversation_history))